In [19]:
#!python -m spacy download en_core_web_md -q

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [38]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import en_core_web_md
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter
import re
nlp = en_core_web_md.load()

# Init

In [10]:
df = pd.read_csv('/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/spam/spam.csv', encoding='ISO-8859-1')

In [11]:
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)

In [13]:
# Initialize the LabelEncoder
le = LabelEncoder()
# Fit label encoder and return encoded labels
encoded_labels = le.fit_transform(df['v1'])
df['encoding'] = encoded_labels

# Pre-processing

 * using the command `str.isalnum` remove all characters from your strings that are not alphanumeric except for whitespaces, and apostrophies.
 * using `str.replace`, `str.lower` and `str.strip` replace double whitespaces with single whitespaces, convert all characters to lowercase and trim starting and finishing whitespaces.
 * using spacy, replace all tokens in your texts with `lemma_` and remove all the stop words.

In [44]:
def clean_text(text):
    # Remove specified characters and symbols
    text = re.sub(r"[.,?!;:*&\-'/()]", "", text)  # Remove listed characters
    text = re.sub(r"\b[a-zA-Z]\b", "", text)  # Remove single letters
    text = re.sub(r"\s+", " ", text)  # Replace multiple spaces with a single space
    text = text.lower().strip()
    
    # Lemmatization and removing stopwords
    text = " ".join([token.lemma_ for token in nlp(text) if (token.lemma_ not in STOP_WORDS) and (token.text not in STOP_WORDS)])
    
    return text

In [45]:
df["v2_clean"] = df["v2"].apply(clean_text)

Are all the v2_clean the same type?

In [42]:
mask = df.v2_clean.apply(lambda x: type(x)==str)
mask.value_counts()

v2_clean
True    5572
Name: count, dtype: int64

let's check most_common words for more preprocessing and prep for tokenizer steps

In [54]:
words = df['v2_clean'].str.lower().str.split(expand=True).stack()

# Count words
word_count = Counter(words)

# Display the most n common words
print(word_count.most_common(20))

[('free', 278), ('ok', 277), ('good', 273), ('send', 270), ('like', 252), ('want', 243), ('ill', 237), ('time', 235), ('day', 234), ('love', 224), ('text', 214), ('tell', 208), ('think', 199), ('need', 185), ('txt', 164), ('home', 162), ('lor', 160), ('reply', 159), ('sorry', 156), ('stop', 155)]


In [53]:
specific_words = {'ur', 'come', 'know', 'm', 's', '2', '4', 'lt#gt', '"'}

# Function to remove specific words from a single text
def remove_specific_words(text):
    return " ".join(word for word in text.split() if word not in specific_words)

# Apply this function to each text in the 'v2_clean' column
df['v2_clean'] = df['v2_clean'].apply(remove_specific_words)


# Tokenize v2_clean

In [67]:
# Collect all words into a single list and then create a set to find unique words
unique_words = set(word for message in df.v2_clean for word in message.split())
# Calculate the total number of unique words
total_unique_words = len(unique_words)

In [68]:
total_words = df.v2_clean.apply(lambda x: len(x.split()))
total_words.sum(), total_unique_words

(41064, 8364)

In [ ]:
tf.keras.preprocessing.text.Tokenizer(num_words=1000, oov_token="out_of_vocab")